## Import Libraries

In [1]:
import gmaps
import pandas as pd

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

Data has already been merged (see 'data story so far.docx'). Read CSV and see dataframe.

In [2]:
data = pd.read_csv("clean_2019_4.csv")

data_df = data

data_df.head()

,Crime ID,Crime type,Last outcome category,Month,Longitude,Latitude,Location,LSOA code,LSOA name,LONG__y,...,LSOA IMD Rank_y,LSOA Income Rank_y,District code (2019),District name (2019),Description,Population,Index of Multiple Deprivation (IMD),Income deprivation- Average score,Income deprivation - Rank of average score,Income - Rank of proportion of LSOAs in most deprived 10% nationally
0,b0854078f33cc4e2ecd7353c40d3a03120063e7d39134b...,Burglary,Investigation complete; no suspect identified,2019-01,-1.850519,52.589275,On or near Crown Lane,E01009417,Birmingham 001A,-1.84982,...,21433,18278,E08000025,Birmingham,Urban with Major Conurbation,1073045.0,7,0.222,7,5
1,b48f19d1ca24bd7e1ef53952c1472314c1af57240df6d5...,Other theft,Investigation complete; no suspect identified,2019-01,-1.850056,52.591108,On or near Walsall Road,E01009417,Birmingham 001A,-1.84982,...,21433,18278,E08000025,Birmingham,Urban with Major Conurbation,1073045.0,7,0.222,7,5
2,2ea61ee241faeac03c5595efc0e3d838d79f20f5831f24...,Public order,Investigation complete; no suspect identified,2019-01,-1.850772,52.588691,On or near Seymour Gardens,E01009417,Birmingham 001A,-1.84982,...,21433,18278,E08000025,Birmingham,Urban with Major Conurbation,1073045.0,7,0.222,7,5
3,0fd56c14bbd052c45e7e9abc1c182c5201e5664a61dc03...,Criminal damage and arson,Investigation complete; no suspect identified,2019-02,-1.850772,52.588691,On or near Seymour Gardens,E01009417,Birmingham 001A,-1.84982,...,21433,18278,E08000025,Birmingham,Urban with Major Conurbation,1073045.0,7,0.222,7,5
4,cadab8dbc07947f5db64c9c0206077758cbbad346611cc...,Criminal damage and arson,Investigation complete; no suspect identified,2019-02,-1.850772,52.588691,On or near Seymour Gardens,E01009417,Birmingham 001A,-1.84982,...,21433,18278,E08000025,Birmingham,Urban with Major Conurbation,1073045.0,7,0.222,7,5


In [4]:
# Create IMD score dataframe
IMD_score_df = pd.read_csv('IMD Score.csv', index_col = 0)
IMD_score_df.dropna(inplace = True) 
IMD_score_df.head()

,LSOA name,LONG__y,LAT_y,LSOA IMD Score
LSOA code,,,,
E01008881,Birmingham 067A,-1.81463,52.4668,41.179
E01008882,Birmingham 066A,-1.82814,52.4663,59.693
E01008883,Birmingham 078A,-1.82302,52.4628,38.636
E01008884,Birmingham 078B,-1.83598,52.4603,44.315
E01008885,Birmingham 076A,-1.80704,52.4612,22.921


In [5]:
# Collect a list of relevant column names and save to a variable
columns = [
    "LSOA code",
    "LSOA name",
    "LSOA IMD Score_y",
    "LONG__y",
    "LAT_y",
    "Crime type"
]

# Filter the data so that only those Burglary's  are in a DataFrame
burglary_df = data_df.loc[data_df["Crime type"] == "Burglary", columns]
# count the relevant crime types and save to a variable 
burglary_counts = burglary_df["LSOA code"].value_counts()
# create a pd dataframe and save to a variable
burglary_counts_df = pd.DataFrame(burglary_counts)
# Convert the column name into "Burglary count"
burglary_counts_df = burglary_counts_df.rename(
    columns={"LSOA code": "Burglary count"})
# Set new index to LSOA code
burglary_counts_df = burglary_counts_df.rename_axis("LSOA code")
# Merge two dataframes using an outer join and save to a variable
merge_df = pd.merge(burglary_counts_df,IMD_score_df, on = "LSOA code",how="outer")
# Drop the null values
merge_df.dropna(inplace = True)
# re-order the columns and save to a variable
burglary_final_df = merge_df[["LSOA name", "LSOA IMD Score",
                            "LONG__y", "LAT_y", "Burglary count"]]
# display the dataframe
burglary_final_df.head()

,LSOA name,LSOA IMD Score,LONG__y,LAT_y,Burglary count
LSOA code,,,,,
E01009642,Coventry 031C,21.685,-1.51165,52.4080,169.0
E01033620,Birmingham 138A,27.122,-1.89970,52.4814,130.0
E01010368,Walsall 030A,61.194,-1.98722,52.5903,94.0
E01010521,Wolverhampton 020F,33.700,-2.12412,52.5910,78.0
E01009114,Birmingham 128D,54.108,-1.94313,52.3959,74.0


In [6]:
mean_value = burglary_final_df["Burglary count"].mean()
mean_value

14.865911799761621

In [7]:
# Store latitude and longitude in locations
locs = burglary_final_df[["LAT_y", "LONG__y"]]

# Fill NaN values and convert to float
incedents = burglary_final_df["Burglary count"].astype(float)

# Convert IMD Score to a list

IMD = IMD_score_df["LSOA IMD Score"].tolist()
# Create a map using coordinates to set markers
marker_locations = IMD_score_df[["LAT_y", "LONG__y"]]

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.symbol_layer(
    marker_locations, fill_color='rgba(0, 0, 255, 0.1)',
    stroke_color='rgba(0, 0, 255, 0.1)', scale=2,
    info_box_content=[f"IMD Score: {score}" for score in IMD]
)

fig.add_layer(markers)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locs, weights=incedents, 
                                 dissipating=False, max_intensity=mean_value*4,
                                 point_radius=.01)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', padding='1px', width='800px'))

In [8]:
# Collect a list of relevant column names and save to a variable
columns = [
    "LSOA code",
    "LSOA name",
    "LSOA IMD Score_y",
    "LONG__y",
    "LAT_y",
    "Crime type"
]

# Filter the data so that only those Bike thefts  are in a DataFrame
bike_theft_df = data_df.loc[data_df["Crime type"] == "Bicycle theft", columns]
# count the relevant crime types and save to a variable 
bike_theft_counts = bike_theft_df["LSOA code"].value_counts()
# create a pd dataframe and save to a variable
bike_theft_counts_df = pd.DataFrame(bike_theft_counts)
# Convert the column name into "Bike theft count"
bike_theft_counts_df = bike_theft_counts_df.rename(
    columns={"LSOA code": "Bike theft count"})
# Set new index to LSOA code
bike_theft_counts_df = bike_theft_counts_df.rename_axis("LSOA code")
# Merge two dataframes using an outer join and save to a variable
merge1_df = pd.merge(bike_theft_counts_df,IMD_score_df, on = "LSOA code",how="outer")
# Drop the null values
merge1_df.dropna(inplace = True)
# re-order the columns and save to a variable
bike_theft_final_df = merge1_df[["LSOA name", "LSOA IMD Score",
                            "LONG__y", "LAT_y", "Bike theft count"]]
# display the dataframe
bike_theft_final_df.head()

,LSOA name,LSOA IMD Score,LONG__y,LAT_y,Bike theft count
LSOA code,,,,,
E01033631,Birmingham 079F,12.334,-1.92120,52.4577,173.0
E01009642,Coventry 031C,21.685,-1.51165,52.4080,155.0
E01033562,Birmingham 087F,28.351,-1.94456,52.4525,66.0
E01033620,Birmingham 138A,27.122,-1.89970,52.4814,50.0
E01010521,Wolverhampton 020F,33.700,-2.12412,52.5910,47.0


In [9]:
mean_value1 = bike_theft_final_df["Bike theft count"].mean()
mean_value1

3.165588615782665

In [10]:
# Store latitude and longitude in locations
locs1 = bike_theft_final_df[["LAT_y", "LONG__y"]]

# Fill NaN values and convert to float
incedents1 = bike_theft_final_df["Bike theft count"].astype(float)

# Convert IMD Score to a list

# IMD = IMD_score_df["LSOA IMD Score"].tolist()
# # Create a map using coordinates to set markers
# marker_locations = IMD_score_df[["LAT_y", "LONG__y"]]

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig1 = gmaps.figure(layout=figure_layout)

markers = gmaps.symbol_layer(
    marker_locations, fill_color='rgba(0, 0, 255, 0.1)',
    stroke_color='rgba(0, 0, 255, 0.1)', scale=2,
    info_box_content=[f"IMD Score: {score}" for score in IMD]
)

fig1.add_layer(markers)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locs1, weights=incedents1, 
                                 dissipating=False, max_intensity=mean_value1*4,
                                 point_radius=.01)


# Add layer
fig1.add_layer(heat_layer)

# Display figure
fig1

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', padding='1px', width='800px'))

In [12]:
# Collect a list of relevant column names and save to a variable
columns = [
    "LSOA code",
    "LSOA name",
    "LSOA IMD Score_y",
    "LONG__y",
    "LAT_y",
    "Crime type"
]

# Filter the data so that only those Bike thefts  are in a DataFrame
asb_df = data_df.loc[data_df["Crime type"] == "Anti-social behaviour", columns]
# count the relevant crime types and save to a variable 
asb_counts = asb_df["LSOA code"].value_counts()
# create a pd dataframe and save to a variable
asb_counts_df = pd.DataFrame(asb_counts)
# Convert the column name into "Anti-social behaviour count"
asb_counts_df = asb_counts_df.rename(
    columns={"LSOA code": "Anti-social behaviour count"})
# Set new index to LSOA code
asb_counts_df = asb_counts_df.rename_axis("LSOA code")
# Merge two dataframes using an outer join and save to a variable
merge2_df = pd.merge(asb_counts_df,IMD_score_df, on = "LSOA code",how="outer")
# Drop the null values
merge2_df.dropna(inplace = True)
# re-order the columns and save to a variable
asb_final_df = merge2_df[["LSOA name", "LSOA IMD Score",
                            "LONG__y", "LAT_y", "Anti-social behaviour count"]]
# display the dataframe
asb_final_df.head()

,LSOA name,LSOA IMD Score,LONG__y,LAT_y,Anti-social behaviour count
LSOA code,,,,,
E01033620,Birmingham 138A,27.122,-1.89970,52.4814,693.0
E01009642,Coventry 031C,21.685,-1.51165,52.4080,385.0
E01033615,Birmingham 135C,25.827,-1.89514,52.4750,296.0
E01010368,Walsall 030A,61.194,-1.98722,52.5903,254.0
E01010521,Wolverhampton 020F,33.700,-2.12412,52.5910,252.0


In [33]:
mean_value2 = asb_final_df["Anti-social behaviour count"].mean()
mean_value2

21.82874251497006

In [34]:
# Store latitude and longitude in locations
locs2 = asb_final_df[["LAT_y", "LONG__y"]]

# Fill NaN values and convert to float
incedents2 = asb_final_df["Anti-social behaviour count"].astype(float)


# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig2 = gmaps.figure(layout=figure_layout)



# Create heat layer
heat_layer = gmaps.heatmap_layer(locs2, weights=incedents2, 
                                 dissipating=False, max_intensity=mean_value2*4,
                                 point_radius=.01)


# Add layer
fig2.add_layer(heat_layer)

# Display figure
fig2

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', padding='1px', width='800px'))

In [15]:
# Collect a list of relevant column names and save to a variable
columns = [
    "LSOA code",
    "LSOA name",
    "LSOA IMD Score_y",
    "LONG__y",
    "LAT_y",
    "Crime type"
]

# Filter the data so that only those Bike thefts  are in a DataFrame
vc_df = data_df.loc[data_df["Crime type"] == "Vehicle crime", columns]
# count the relevant crime types and save to a variable 
vc_counts = vc_df["LSOA code"].value_counts()
# create a pd dataframe and save to a variable
vc_counts_df = pd.DataFrame(vc_counts)
# Convert the column name into "Vehicle crime count"
vc_counts_df = vc_counts_df.rename(
    columns={"LSOA code": "Vehicle crime count"})
# Set new index to LSOA code
vc_counts_df = vc_counts_df.rename_axis("LSOA code")
# Merge two dataframes using an outer join and save to a variable
merge3_df = pd.merge(vc_counts_df,IMD_score_df, on = "LSOA code",how="outer")
# Drop the null values
merge3_df.dropna(inplace = True)
# re-order the columns and save to a variable
vc_final_df = merge3_df[["LSOA name", "LSOA IMD Score",
                            "LONG__y", "LAT_y", "Vehicle crime count"]]
# display the dataframe
vc_final_df.head()

,LSOA name,LSOA IMD Score,LONG__y,LAT_y,Vehicle crime count
LSOA code,,,,,
E01010109,Solihull 009A,18.570,-1.72677,52.4510,238
E01033561,Birmingham 050E,26.700,-1.88445,52.4845,194
E01009378,Birmingham 135A,43.802,-1.88735,52.4707,180
E01009200,Birmingham 139A,54.122,-1.87489,52.4764,168
E01010521,Wolverhampton 020F,33.700,-2.12412,52.5910,140


In [35]:
mean_value3 = vc_final_df["Vehicle crime count"].mean()
mean_value3

19.24642857142857

In [36]:
# Store latitude and longitude in locations
locs3 = vc_final_df[["LAT_y", "LONG__y"]]

# Fill NaN values and convert to float
incedents3 = vc_final_df["Vehicle crime count"].astype(float)


# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig3 = gmaps.figure(layout=figure_layout)



# Create heat layer
heat_layer = gmaps.heatmap_layer(locs3, weights=incedents3, 
                                 dissipating=False, max_intensity=mean_value3*4,
                                 point_radius=.01)


# Add layer
fig3.add_layer(heat_layer)

# Display figure
fig3

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', padding='1px', width='800px'))